In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Settings

In [2]:
import os 
import pandas as pd

RecName_List = [
    'P', 'EC', 
    'Diag', 'A1C', 'Smoking',
    'PN', 'PNSect', 'PNSectSent',
 ]

# RecName_List
folder = 'data/Simulations/'
file_list = [os.path.join(folder, i) for i in RecName_List]
file_list

['data/Simulations/P',
 'data/Simulations/EC',
 'data/Simulations/Diag',
 'data/Simulations/A1C',
 'data/Simulations/Smoking',
 'data/Simulations/PN',
 'data/Simulations/PNSect',
 'data/Simulations/PNSectSent']

In [3]:
# RecName_List
folder = 'data/Simulations/'
RecName_2_File = {i: os.path.join(folder, i) + '.csv' for i in RecName_List}
RecName_2_File

{'P': 'data/Simulations/P.csv',
 'EC': 'data/Simulations/EC.csv',
 'Diag': 'data/Simulations/Diag.csv',
 'A1C': 'data/Simulations/A1C.csv',
 'Smoking': 'data/Simulations/Smoking.csv',
 'PN': 'data/Simulations/PN.csv',
 'PNSect': 'data/Simulations/PNSect.csv',
 'PNSectSent': 'data/Simulations/PNSectSent.csv'}

# Rec `P`

In [4]:
RecName = 'P'

# load all records
df = pd.read_csv(RecName_2_File[RecName])
df

,PID,age,basicInfo
0,P0,52,Male&B
1,P1,21,Male&B
2,P2,23,Female&B
3,P3,77,Female&B
4,P4,23,Male&A
5,P5,73,Female&D
6,P6,34,Female&A
7,P7,76,Male&C


In [5]:
prefix_ids = []
focal_ids = ['PID']
field_list = ['basicInfo', 'age', 
              # 'Encounter'
             ]

## Field `basicInfo`

In [7]:
# (a) field
field = 'basicInfo'

# (b) get field-focused df
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,basicInfo
0,P0,Male&B
1,P1,Male&B
2,P2,Female&B
3,P3,Female&B
4,P4,Male&A
5,P5,Female&D
6,P6,Female&A
7,P7,Male&C


### Grain `basicInfoDftGrn`

In [11]:
import itertools

############## <s>
def generate_grain_vocab_info(s):
    value_list = itertools.chain(*s.to_list())
    d = pd.Series(value_list).value_counts().to_dict()
    v2freq = {}
    if '_missing' in d:
        v2freq['_missing'] = d.pop('_missing') # I should have some prefix here. 
    for k, v in d.items(): v2freq[k] = v

    idx2v = {idx: v for idx, v in enumerate(v2freq.keys())}
    v2idx = {v:k for k, v in idx2v.items()}
    
    return {'idx2v': idx2v, 'v2idx': v2idx, 'v2freq': v2freq}
############## <e>


In [12]:
############## <s> (exported from modules)
def get_basicInfoGrn_list(x):
    if pd.isna(x):
        return ['_missing']
    else:
        return x.split('&')

GrnName_2_GrnFun[field2grain] = get_basicInfoGrn_list
############## <e>

In [10]:
# (c) grain
grain = 'basicInfoDftGrn'

# (d): field2grain str func
field2grain = '-'.join([field, grain])

grain_str_fn = GrnName_2_GrnFun[field2grain]

# (e) field2grain result
dfx[field2grain] = dfx[field].apply(grain_str_fn)

# (f) grain_dict
Vocab = generate_grain_vocab_info(dfx[field2grain]) # load from disk
v2idx = Vocab['v2idx']

# (g) mapping grain_str to grain_idx
dfx[field2grain] = dfx[field2grain].apply(lambda x: [v2idx[i] for i in x])
dfx

,PID,basicInfo,basicInfo-basicInfoDftGrn
0,P0,Male&B,"[0, 1]"
1,P1,Male&B,"[0, 1]"
2,P2,Female&B,"[2, 1]"
3,P3,Female&B,"[2, 1]"
4,P4,Male&A,"[0, 3]"
5,P5,Female&D,"[2, 4]"
6,P6,Female&A,"[2, 3]"
7,P7,Male&C,"[0, 5]"


## Field `Age`

In [10]:
# (a) field
field = 'age'

# (b) get field-focused df
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,age
0,P0,52
1,P1,21
2,P2,23
3,P3,77
4,P4,23
5,P5,73
6,P6,34
7,P7,76


### Grain `AgeDftGrn`

In [11]:
import numpy as np

def func_convert_Num_factory(start, end, Min, Max, scale):
    def func_convert_NUMgrn(v):
    
        miss, more, less, bottom, top = 0, 0, 0, 0, 0; base = 1
        if pd.isna(v) == True:
            miss = 1; base = 0
            num_embed = int((end - start) / scale)
            x = np.zeros(num_embed + 1)# .astype(int)
            x = list(x)
            li = [miss, more, less, bottom, top, base] + x
            num_embed = num_embed + 7
        else:
            if type(v) == str:
                if v[0] not in '<>-':
                    v = float(v)
                elif v[0] == '<':
                    less = 1
                    v = float(v[1:])
                elif v[0] == '>':
                    more = 1
                    v = float(v[1:])
                
            if v < start: 
                bottom = (start - v) / (start - Min)
                base = 0
                v = start
                
            elif v > end: 
                top = (v-end) / (Max-end)
                v= end

            num_embed = int((end - start) / scale)
            x = np.zeros(num_embed + 1)# .astype(f)
            num = int((v - start) / scale)
            # print(num)
            x[:num] = 1
            # print(v, start, scale,  v - start, num)
            # print( (v-start) - num * scale)
            x[num] = ((v-start) - num * scale) / scale
            x = x.round(4)
            # print((v - start) % scale, v, start, scale)
            # print(x[num])
            # print(x)
            x = list(x) # value part

            li = [miss, more, less, bottom, top, base] + x
            num_embed = num_embed + 7
        return li
    
    return func_convert_NUMgrn


# from ._base import func_convert_Num_factory



In [12]:
# (c) grain
grain = 'AgeDftGrn'

# (d): field2grain str func
field2grain = '-'.join([field, grain])

############## <s> (exported from modules)
start, end, Min, Max, scale = [18, 80, 1, 120, 5]
func_convert_AGEwgt = func_convert_Num_factory(start, end, Min, Max, scale)

GrnName_2_GrnFun[field2grain] = func_convert_AGEwgt
############## <e>
grain_str_fn = GrnName_2_GrnFun[field2grain]

# (e) field2grain result
dfx[field2grain] = dfx[field].apply(grain_str_fn)

dfx

,PID,age,age-AgeDftGrn
0,P0,52,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
1,P1,21,"[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0...."
2,P2,23,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0...."
3,P3,77,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
4,P4,23,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0...."
5,P5,73,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
6,P6,34,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0...."
7,P7,76,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."


In [13]:
# dfx['age-AgeDftGrn'].apply(len)

# Rec `EC`

In [14]:
RecName = 'EC'

# load all records
df = pd.read_csv(RecName_2_File[RecName])
df['DT_min'] = pd.to_datetime(df['DT_min'])
df['DT_max'] = pd.to_datetime(df['DT_max'])
df['Duration'] = (df['DT_max'] - df['DT_min']).dt.days
df

,PID,ECID,DT_min,DT_max,BasicInfo,Duration
0,P0,EC18,2023-03-18,2023-04-16,B&W,29
1,P0,EC140,2023-03-19,2023-04-05,A&X,17
2,P0,EC23,2023-03-19,2023-04-16,D&W,28
3,P0,EC141,2023-03-20,2023-04-16,C&Z,27
4,P0,EC142,2023-03-20,2023-04-15,D&Z,26
...,...,...,...,...,...,...
220,P6,EC15,2023-03-21,2023-04-09,B&W,19
221,P6,EC93,2023-03-23,2023-04-04,C&Z,12
222,P6,EC195,2023-03-25,2023-03-25,C&Z,0
223,P6,EC173,2023-04-05,2023-04-13,D&Y,8


In [15]:
prefix_ids = ['PID']
focal_ids = ['ECID']
field_list = ['DT_min', 'DT_max', 
              'BasicInfo',
             ]

## Field `DT_min`

In [16]:
# (a) field
field = 'DT_min'

# (b) get field-focused df
dfx = df[prefix_ids + focal_ids + [field]].reset_index(drop = True)
dfx

,PID,ECID,DT_min
0,P0,EC18,2023-03-18
1,P0,EC140,2023-03-19
2,P0,EC23,2023-03-19
3,P0,EC141,2023-03-20
4,P0,EC142,2023-03-20
...,...,...,...
220,P6,EC15,2023-03-21
221,P6,EC93,2023-03-23
222,P6,EC195,2023-03-25
223,P6,EC173,2023-04-05


### Grain `DTdftGrn`

In [17]:
import pandas as pd


def load_DTgrn_Dictionary():
    Y = ['Yx'] + [f'Y{i}' for i in range(2010, 2030)]
    M = ['Mx'] +[f'M{i}' for i in range(1, 13)]
    D = ['Dx'] + [f'D{i}' for i in range(1, 32)]
    S = ['Sx'] +[f'S{i}' for i in range(1, 5)]
    W = ['Wx'] + [f'W{i}' for i in range(1, 8)]
    WY = ['WYx'] + [f'WY{i}' for i in range(1, 56)]
    H = ['Hx'] + [f'H{i}' for i in range(0, 24)]
    value_list = ['_missing'] + Y + M + D + S + W + WY + H 
    v2freq = {k:0 for k in value_list}
    idx2v = {idx: v for idx, v in enumerate(v2freq.keys())}
    v2idx = {v:k for k, v in idx2v.items()}
    DT_v2idx2freq = {'idx2v': idx2v, 'v2idx': v2idx, 'v2freq': v2freq}
    return DT_v2idx2freq


def func_convert_DTgrn(x):
    if pd.isna(x):
        nan = 'x'
        return f'Y{nan} M{nan} D{nan} S{nan} W{nan} WY{nan} H{nan}'.split()
    
    # dfx['DT'] = pd.to_datetime(dfx['DT'])
    DT = x
    year = DT.year
    month = DT.month
    date = DT.day
    season = int(month / 4)+ 1
    weekday = DT.weekday() + 1
    weekofyear = int(DT.isocalendar()[1]) + 1
    hour = DT.hour
    
    return f'Y{year} M{month} D{date} S{season} W{weekday} WY{weekofyear} H{hour}'.split()
    

In [18]:
# (c) grain
grain = 'DTDftGrn'

# (d): field2grain str func
field2grain = '-'.join([field, grain])

############## <s> (exported from modules)
GrnName_2_GrnFun[field2grain] = func_convert_DTgrn
############## <e>
grain_str_fn = GrnName_2_GrnFun[field2grain]

# (e) field2grain result
dfx[field2grain] = dfx[field].apply(grain_str_fn)

Vocab = load_DTgrn_Dictionary()
v2idx = Vocab['v2idx']

# (g) mapping grain_str to grain_idx
dfx[field2grain] = dfx[field2grain].apply(lambda x: [v2idx[i] for i in x])
dfx


,PID,ECID,DT_min,DT_min-DTDftGrn
0,P0,EC18,2023-03-18,"[15, 25, 53, 68, 78, 92, 137]"
1,P0,EC140,2023-03-19,"[15, 25, 54, 68, 79, 92, 137]"
2,P0,EC23,2023-03-19,"[15, 25, 54, 68, 79, 92, 137]"
3,P0,EC141,2023-03-20,"[15, 25, 55, 68, 73, 93, 137]"
4,P0,EC142,2023-03-20,"[15, 25, 55, 68, 73, 93, 137]"
...,...,...,...,...
220,P6,EC15,2023-03-21,"[15, 25, 56, 68, 74, 93, 137]"
221,P6,EC93,2023-03-23,"[15, 25, 58, 68, 76, 93, 137]"
222,P6,EC195,2023-03-25,"[15, 25, 60, 68, 78, 93, 137]"
223,P6,EC173,2023-04-05,"[15, 26, 40, 69, 75, 95, 137]"


## Field `Duration`

In [19]:
# (a) field
field = 'Duration'

# (b) get field-focused df
dfx = df[prefix_ids + focal_ids + [field]].reset_index(drop = True)
dfx

,PID,ECID,Duration
0,P0,EC18,29
1,P0,EC140,17
2,P0,EC23,28
3,P0,EC141,27
4,P0,EC142,26
...,...,...,...
220,P6,EC15,19
221,P6,EC93,12
222,P6,EC195,0
223,P6,EC173,8


### Grain `DurationDftGrn`

In [20]:
# (c) grain
grain = 'DurationDftGrn'

# (d): field2grain str func
field2grain = '-'.join([field, grain])

############## <s> (exported from modules)
start, end, Min, Max, scale = [0, 90, 0, 120, 5]
func_convert_AGEwgt = func_convert_Num_factory(start, end, Min, Max, scale)

GrnName_2_GrnFun[field2grain] = func_convert_AGEwgt
############## <e>
grain_str_fn = GrnName_2_GrnFun[field2grain]

# (e) field2grain result
dfx[field2grain] = dfx[field].apply(grain_str_fn)

dfx

,PID,ECID,Duration,Duration-DurationDftGrn
0,P0,EC18,29,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 0...."
1,P0,EC140,17,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.4, 0.0, 0...."
2,P0,EC23,28,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 0...."
3,P0,EC141,27,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 0...."
4,P0,EC142,26,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 0...."
...,...,...,...,...
220,P6,EC15,19,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.8, 0.0, 0...."
221,P6,EC93,12,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 0.4, 0.0, 0.0, 0...."
222,P6,EC195,0,"[0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
223,P6,EC173,8,"[0, 0, 0, 0, 0, 1, 1.0, 0.6, 0.0, 0.0, 0.0, 0...."


## Field `Basic`

In [21]:
# (a) field
field = 'BasicInfo'

# (b) get field-focused df
dfx = df[prefix_ids + focal_ids + [field]].reset_index(drop = True)
dfx

,PID,ECID,BasicInfo
0,P0,EC18,B&W
1,P0,EC140,A&X
2,P0,EC23,D&W
3,P0,EC141,C&Z
4,P0,EC142,D&Z
...,...,...,...
220,P6,EC15,B&W
221,P6,EC93,C&Z
222,P6,EC195,C&Z
223,P6,EC173,D&Y


### Grain `BasicDftGrn`

In [22]:
import itertools

# (c) grain
grain = 'BasicDftGrn'

# (d): field2grain str func
field2grain = '-'.join([field, grain])
full_recfldgrn_name = RecName+':' + field2grain


############## <s> (exported from modules)
def get_basicInfoGrn_list(x):
    if pd.isna(x):
        return ['_missing']
    else:
        return x.split('&')

GrnName_2_GrnFun[field2grain] = get_basicInfoGrn_list
############## <e>
grain_str_fn = GrnName_2_GrnFun[field2grain]

# (e) field2grain result
dfx[field2grain] = dfx[field].apply(grain_str_fn)

# (f) grain_dict
Vocab = generate_grain_vocab_info(dfx[field2grain]) # load from disk
v2idx = Vocab['v2idx']

# (g) mapping grain_str to grain_idx
dfx[field2grain] = dfx[field2grain].apply(lambda x: [v2idx[i] for i in x])
dfx = dfx.rename(columns = {field2grain: full_recfldgrn_name})
dfx

,PID,ECID,BasicInfo,EC:BasicInfo-BasicDftGrn
0,P0,EC18,B&W,"[6, 4]"
1,P0,EC140,A&X,"[2, 7]"
2,P0,EC23,D&W,"[1, 4]"
3,P0,EC141,C&Z,"[5, 0]"
4,P0,EC142,D&Z,"[1, 0]"
...,...,...,...,...
220,P6,EC15,B&W,"[6, 4]"
221,P6,EC93,C&Z,"[5, 0]"
222,P6,EC195,C&Z,"[5, 0]"
223,P6,EC173,D&Y,"[1, 3]"


In [23]:
df_Rec = dfx[prefix_ids + [full_recfldgrn_name]]
df_Rec

,PID,EC:BasicInfo-BasicDftGrn
0,P0,"[6, 4]"
1,P0,"[2, 7]"
2,P0,"[1, 4]"
3,P0,"[5, 0]"
4,P0,"[1, 0]"
...,...,...
220,P6,"[6, 4]"
221,P6,"[5, 0]"
222,P6,"[5, 0]"
223,P6,"[1, 3]"


In [24]:
def get_compressed_df(df, full_recfldgrn_name, prefix_ids):
    df_rec = df
    CompressParent_ID = prefix_ids[-1]
    prefix_ids_new = prefix_ids[:-1]

    df_rec_new = pd.DataFrame(df_rec.groupby(CompressParent_ID).apply(lambda x: x.to_dict('list')).to_list())

    for col in prefix_ids: 
        df_rec_new[col] = df_rec_new[col].apply(lambda x: x[0])
        
    full_recfldgrn_name_new = CompressParent_ID.replace('ID', '') + '-' + full_recfldgrn_name
    df_rec_new = df_rec_new.rename(columns = {full_recfldgrn_name: full_recfldgrn_name_new})
    df_rec_new = df_rec_new[prefix_ids + [full_recfldgrn_name_new]]
    return df_rec_new, full_recfldgrn_name_new, prefix_ids_new



In [25]:
dfx

,PID,ECID,BasicInfo,EC:BasicInfo-BasicDftGrn
0,P0,EC18,B&W,"[6, 4]"
1,P0,EC140,A&X,"[2, 7]"
2,P0,EC23,D&W,"[1, 4]"
3,P0,EC141,C&Z,"[5, 0]"
4,P0,EC142,D&Z,"[1, 0]"
...,...,...,...,...
220,P6,EC15,B&W,"[6, 4]"
221,P6,EC93,C&Z,"[5, 0]"
222,P6,EC195,C&Z,"[5, 0]"
223,P6,EC173,D&Y,"[1, 3]"


In [26]:

print(full_recfldgrn_name)
print(prefix_ids)

df_p, full_recfldgrn_name_new, prefix_ids_new = get_compressed_df(df_Rec, full_recfldgrn_name, prefix_ids)

print(full_recfldgrn_name_new)
print(prefix_ids_new)

df_p

EC:BasicInfo-BasicDftGrn
['PID']
P-EC:BasicInfo-BasicDftGrn
[]


,PID,P-EC:BasicInfo-BasicDftGrn
0,P0,"[[6, 4], [2, 7], [1, 4], [5, 0], [1, 0], [6, 4..."
1,P1,"[[1, 0], [1, 7], [1, 7], [6, 0], [1, 4], [6, 4..."
2,P2,"[[2, 0], [1, 0], [2, 0], [5, 4], [6, 0], [1, 4..."
3,P3,"[[6, 7], [5, 4], [6, 4], [6, 3], [2, 4], [1, 7..."
4,P4,"[[6, 0], [2, 3], [5, 4], [2, 0], [2, 4], [5, 3..."
5,P5,"[[5, 7], [1, 4], [2, 4], [1, 4], [2, 3], [2, 3..."
6,P6,"[[6, 0], [1, 0], [5, 3], [6, 4], [5, 0], [5, 0..."
7,P7,"[[2, 4]]"


# Rec `PN`

In [27]:
RecName = 'PN'

# load all records
df = pd.read_csv(RecName_2_File[RecName])
df['DT'] = pd.to_datetime(df['DT'])
df

,PID,ECID,PNID,DT
0,P0,EC18,71800,2023-03-18
1,P0,EC13,70411,2023-03-26
2,P0,EC16,70566,2023-03-31
3,P0,EC14,71802,2023-04-01
4,P0,EC19,71667,2023-04-01
5,P0,EC20,70410,2023-04-07
6,P0,EC12,71801,2023-04-08
7,P0,EC21,72374,2023-04-12
8,P1,EC2,70851,2023-03-26
9,P1,EC0,71666,2023-04-03


In [28]:
prefix_ids = ['PID', 'ECID']
focal_ids = ['PNID']
field_list = ['DT',
             ]

## Field `DT`

In [29]:
# (a) field
field = 'DT'

# (b) get field-focused df
dfx = df[prefix_ids + focal_ids + [field]].reset_index(drop = True)
dfx

,PID,ECID,PNID,DT
0,P0,EC18,71800,2023-03-18
1,P0,EC13,70411,2023-03-26
2,P0,EC16,70566,2023-03-31
3,P0,EC14,71802,2023-04-01
4,P0,EC19,71667,2023-04-01
5,P0,EC20,70410,2023-04-07
6,P0,EC12,71801,2023-04-08
7,P0,EC21,72374,2023-04-12
8,P1,EC2,70851,2023-03-26
9,P1,EC0,71666,2023-04-03


### Grain `DTdftGrn`

In [30]:
# (c) grain
grain = 'DTDftGrn'

# (d): field2grain str func
field2grain = '-'.join([field, grain])
full_recfldgrn_name = RecName+':' + field2grain


############## <s> (exported from modules)
GrnName_2_GrnFun[field2grain] = func_convert_DTgrn
############## <e>
grain_str_fn = GrnName_2_GrnFun[field2grain]

# (e) field2grain result
dfx[field2grain] = dfx[field].apply(grain_str_fn)

Vocab = load_DTgrn_Dictionary()
v2idx = Vocab['v2idx']

# (g) mapping grain_str to grain_idx
dfx[field2grain] = dfx[field2grain].apply(lambda x: [v2idx[i] for i in x])
dfx = dfx.rename(columns = {field2grain: full_recfldgrn_name})
dfx


,PID,ECID,PNID,DT,PN:DT-DTDftGrn
0,P0,EC18,71800,2023-03-18,"[15, 25, 53, 68, 78, 92, 137]"
1,P0,EC13,70411,2023-03-26,"[15, 25, 61, 68, 79, 93, 137]"
2,P0,EC16,70566,2023-03-31,"[15, 25, 66, 68, 77, 94, 137]"
3,P0,EC14,71802,2023-04-01,"[15, 26, 36, 69, 78, 94, 137]"
4,P0,EC19,71667,2023-04-01,"[15, 26, 36, 69, 78, 94, 137]"
5,P0,EC20,70410,2023-04-07,"[15, 26, 42, 69, 77, 95, 137]"
6,P0,EC12,71801,2023-04-08,"[15, 26, 43, 69, 78, 95, 137]"
7,P0,EC21,72374,2023-04-12,"[15, 26, 47, 69, 75, 96, 137]"
8,P1,EC2,70851,2023-03-26,"[15, 25, 61, 68, 79, 93, 137]"
9,P1,EC0,71666,2023-04-03,"[15, 26, 38, 69, 73, 95, 137]"


In [31]:
df_Rec = dfx[prefix_ids + [full_recfldgrn_name]]
df_Rec

,PID,ECID,PN:DT-DTDftGrn
0,P0,EC18,"[15, 25, 53, 68, 78, 92, 137]"
1,P0,EC13,"[15, 25, 61, 68, 79, 93, 137]"
2,P0,EC16,"[15, 25, 66, 68, 77, 94, 137]"
3,P0,EC14,"[15, 26, 36, 69, 78, 94, 137]"
4,P0,EC19,"[15, 26, 36, 69, 78, 94, 137]"
5,P0,EC20,"[15, 26, 42, 69, 77, 95, 137]"
6,P0,EC12,"[15, 26, 43, 69, 78, 95, 137]"
7,P0,EC21,"[15, 26, 47, 69, 75, 96, 137]"
8,P1,EC2,"[15, 25, 61, 68, 79, 93, 137]"
9,P1,EC0,"[15, 26, 38, 69, 73, 95, 137]"


In [32]:

print(full_recfldgrn_name)
print(prefix_ids)

df_p, full_recfldgrn_name_new, prefix_ids_new = get_compressed_df(df_Rec, full_recfldgrn_name, prefix_ids)

print(full_recfldgrn_name_new)
print(prefix_ids_new)

df_p

PN:DT-DTDftGrn
['PID', 'ECID']
EC-PN:DT-DTDftGrn
['PID']


,PID,ECID,EC-PN:DT-DTDftGrn
0,P1,EC0,"[[15, 26, 38, 69, 73, 95, 137]]"
1,P3,EC1,"[[15, 26, 46, 69, 74, 96, 137]]"
2,P4,EC10,"[[15, 26, 49, 69, 77, 96, 137]]"
3,P5,EC11,"[[15, 26, 50, 69, 78, 96, 137]]"
4,P0,EC12,"[[15, 26, 43, 69, 78, 95, 137]]"
5,P0,EC13,"[[15, 25, 61, 68, 79, 93, 137]]"
6,P0,EC14,"[[15, 26, 36, 69, 78, 94, 137]]"
7,P6,EC15,"[[15, 25, 66, 68, 77, 94, 137]]"
8,P0,EC16,"[[15, 25, 66, 68, 77, 94, 137]]"
9,P7,EC17,"[[15, 25, 62, 68, 73, 94, 137]]"


In [33]:
df_Rec = df_p
prefix_ids = prefix_ids_new
full_recfldgrn_name = full_recfldgrn_name_new


print(full_recfldgrn_name)
print(prefix_ids)

df_p, full_recfldgrn_name_new, prefix_ids_new = get_compressed_df(df_Rec, full_recfldgrn_name, prefix_ids)

print(full_recfldgrn_name_new)
print(prefix_ids_new)

df_p

EC-PN:DT-DTDftGrn
['PID']
P-EC-PN:DT-DTDftGrn
[]


,PID,P-EC-PN:DT-DTDftGrn
0,P0,"[[[15, 26, 43, 69, 78, 95, 137]], [[15, 25, 61..."
1,P1,"[[[15, 26, 38, 69, 73, 95, 137]], [[15, 25, 61..."
2,P2,"[[[15, 26, 36, 69, 78, 94, 137]], [[15, 25, 65..."
3,P3,"[[[15, 26, 46, 69, 74, 96, 137]], [[15, 25, 54..."
4,P4,"[[[15, 26, 49, 69, 77, 96, 137]], [[15, 25, 58..."
5,P5,"[[[15, 26, 50, 69, 78, 96, 137]]]"
6,P6,"[[[15, 25, 66, 68, 77, 94, 137]]]"
7,P7,"[[[15, 25, 62, 68, 73, 94, 137]]]"


# Rec `PNSect`

In [34]:
RecName = 'PNSect'

# load all records
df = pd.read_csv(RecName_2_File[RecName])
df

,PID,ECID,PNID,PNSectID,SectName
0,P0,EC21,72374,140,History of Present Illness
1,P0,EC21,72374,142,Physical Exam
2,P0,EC21,72374,143,Assessment/Plan
3,P0,EC21,72374,144,Patient Instructions
4,P0,EC21,72374,145,Problem List/Past Medical History
...,...,...,...,...,...
277,P7,EC17,435,44,Problem List/Past Medical History
278,P7,EC17,435,43,Assessment/Plan
279,P7,EC17,435,42,Physical Exam
280,P7,EC17,435,41,Review of Systems


In [35]:
prefix_ids = ['PID', 'ECID', 'PNID']
focal_ids = ['PNSectID']
field_list = ['SectName',
             ]

## Field `SectName`

In [36]:
# (a) field
field = 'SectName'

# (b) get field-focused df
dfx = df[prefix_ids + focal_ids + [field]].reset_index(drop = True)
dfx

,PID,ECID,PNID,PNSectID,SectName
0,P0,EC21,72374,140,History of Present Illness
1,P0,EC21,72374,142,Physical Exam
2,P0,EC21,72374,143,Assessment/Plan
3,P0,EC21,72374,144,Patient Instructions
4,P0,EC21,72374,145,Problem List/Past Medical History
...,...,...,...,...,...
277,P7,EC17,435,44,Problem List/Past Medical History
278,P7,EC17,435,43,Assessment/Plan
279,P7,EC17,435,42,Physical Exam
280,P7,EC17,435,41,Review of Systems


### Grain `SectNmDftGrn`

In [37]:
import itertools

# (c) grain
grain = 'SectNmDftGrn'

# (d): field2grain str func
field2grain = '-'.join([field, grain])
full_recfldgrn_name = RecName+':' + field2grain


############## <s> (exported from modules)
def get_SectNmDftGrn_list(x):
    if pd.isna(x):
        return ['_missing']
    else:
        return x.split('/')

GrnName_2_GrnFun[field2grain] = get_SectNmDftGrn_list
############## <e>
grain_str_fn = GrnName_2_GrnFun[field2grain]

# (e) field2grain result
dfx[field2grain] = dfx[field].apply(grain_str_fn)

# (f) grain_dict
Vocab = generate_grain_vocab_info(dfx[field2grain]) # load from disk
v2idx = Vocab['v2idx']

# (g) mapping grain_str to grain_idx
dfx[field2grain] = dfx[field2grain].apply(lambda x: [v2idx[i] for i in x])
dfx = dfx.rename(columns = {field2grain: full_recfldgrn_name})
dfx

,PID,ECID,PNID,PNSectID,SectName,PNSect:SectName-SectNmDftGrn
0,P0,EC21,72374,140,History of Present Illness,[0]
1,P0,EC21,72374,142,Physical Exam,[3]
2,P0,EC21,72374,143,Assessment/Plan,"[10, 9]"
3,P0,EC21,72374,144,Patient Instructions,[15]
4,P0,EC21,72374,145,Problem List/Past Medical History,"[8, 7]"
...,...,...,...,...,...,...
277,P7,EC17,435,44,Problem List/Past Medical History,"[8, 7]"
278,P7,EC17,435,43,Assessment/Plan,"[10, 9]"
279,P7,EC17,435,42,Physical Exam,[3]
280,P7,EC17,435,41,Review of Systems,[14]


In [38]:
# size_prefix_ids = len(prefix_ids)

prefix_ids_c = prefix_ids.copy()
full_recfldgrn_name_c = full_recfldgrn_name

In [39]:
df_Rec = dfx[prefix_ids_c + [full_recfldgrn_name_c]]
for i in range(len(prefix_ids_c)):
    
    print(prefix_ids_c, full_recfldgrn_name_c)
    df_Rec, full_recfldgrn_name_c, prefix_ids_c = get_compressed_df(df_Rec, 
                                                                    full_recfldgrn_name_c, 
                                                                    prefix_ids_c)
    print(prefix_ids_c, full_recfldgrn_name_c, '\n')
    
    
    
df_p = df_Rec
df_p

['PID', 'ECID', 'PNID'] PNSect:SectName-SectNmDftGrn
['PID', 'ECID'] PN-PNSect:SectName-SectNmDftGrn 

['PID', 'ECID'] PN-PNSect:SectName-SectNmDftGrn
['PID'] EC-PN-PNSect:SectName-SectNmDftGrn 

['PID'] EC-PN-PNSect:SectName-SectNmDftGrn
[] P-EC-PN-PNSect:SectName-SectNmDftGrn 



,PID,P-EC-PN-PNSect:SectName-SectNmDftGrn
0,P0,"[[[[11], [12], [0], [14], [3], [10, 9], [15], ..."
1,P1,"[[[[3], [14], [0], [12], [10, 9], [15], [16], ..."
2,P2,"[[[[11], [17], [13], [2], [6], [4], [16], [3],..."
3,P3,"[[[[3], [18], [10, 9], [15], [4], [8, 7], [1, ..."
4,P4,"[[[[11], [2], [6], [4], [1, 5], [8, 7], [15], ..."
5,P5,"[[[[11], [8, 7], [6], [4], [1, 5], [15], [10, ..."
6,P6,"[[[[6], [4], [1, 5], [8, 7], [15], [10, 9], [1..."
7,P7,"[[[[12], [11], [13], [2], [6], [4], [1, 5], [8..."


# Rec `PNSectSent`

In [59]:
RecName = 'PNSectSent'

# load all records
df = pd.read_csv(RecName_2_File[RecName])
df

,PID,ECID,PNID,PNSectID,SentID,Sentence
0,P0,EC21,72374,149,1293,Later available sure scientist water out less.
1,P0,EC21,72374,145,1272,Wear relationship down surface section.
2,P0,EC21,72374,145,1271,Concern painting able than.
3,P0,EC21,72374,145,1270,Quite together just fund field.
4,P0,EC21,72374,145,1269,Owner PM building win cultural agent forward.
...,...,...,...,...,...,...
2582,P7,EC17,435,43,372,Man artist food area character.
2583,P7,EC17,435,43,373,Language very back strategy.
2584,P7,EC17,435,43,374,Feel budget tough edge read collection however...
2585,P7,EC17,435,39,310,Car true daughter guy strategy.


In [60]:
prefix_ids = ['PID', 'ECID', 'PNID', 'PNSectID']
focal_ids = ['SentID']
field_list = ['Sentence',
             ]

## Field `Sentence`

In [61]:
# (a) field
field = 'Sentence'

# (b) get field-focused df
dfx = df[prefix_ids + focal_ids + [field]].reset_index(drop = True)
dfx

,PID,ECID,PNID,PNSectID,SentID,Sentence
0,P0,EC21,72374,149,1293,Later available sure scientist water out less.
1,P0,EC21,72374,145,1272,Wear relationship down surface section.
2,P0,EC21,72374,145,1271,Concern painting able than.
3,P0,EC21,72374,145,1270,Quite together just fund field.
4,P0,EC21,72374,145,1269,Owner PM building win cultural agent forward.
...,...,...,...,...,...,...
2582,P7,EC17,435,43,372,Man artist food area character.
2583,P7,EC17,435,43,373,Language very back strategy.
2584,P7,EC17,435,43,374,Feel budget tough edge read collection however...
2585,P7,EC17,435,39,310,Car true daughter guy strategy.


### Grain `TkLLMGrn`

In [62]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


In [63]:
import itertools

# (c) grain
grain = 'TkLLMGrn'

# (d): field2grain str func
field2grain = '-'.join([field, grain])
full_recfldgrn_name = RecName+':' + field2grain


############## <s> (exported from modules)
def get_TkLLMGrnIdx_list(x, tokenizer):
    if pd.isna(x):
        return ['_missing']
    else:
        return tokenizer(x)['input_ids']

# (e) field2grain result
grain_idx_fn = get_TkLLMGrnIdx_list
dfx[field2grain] = dfx[field].apply(lambda x: grain_idx_fn(x, tokenizer))

dfx = dfx.rename(columns = {field2grain: full_recfldgrn_name})
dfx

,PID,ECID,PNID,PNSectID,SentID,Sentence,PNSectSent:Sentence-TkLLMGrn
0,P0,EC21,72374,149,1293,Later available sure scientist water out less.,"[101, 2611, 1907, 1612, 7482, 1447, 1149, 1750..."
1,P0,EC21,72374,145,1272,Wear relationship down surface section.,"[101, 1284, 1813, 2398, 1205, 2473, 2237, 119,..."
2,P0,EC21,72374,145,1271,Concern painting able than.,"[101, 16752, 14840, 1179, 3504, 1682, 1190, 11..."
3,P0,EC21,72374,145,1270,Quite together just fund field.,"[101, 22597, 1487, 1198, 5841, 1768, 119, 102]"
4,P0,EC21,72374,145,1269,Owner PM building win cultural agent forward.,"[101, 13432, 1200, 14123, 1459, 1782, 3057, 36..."
...,...,...,...,...,...,...,...
2582,P7,EC17,435,43,372,Man artist food area character.,"[101, 2268, 2360, 2094, 1298, 1959, 119, 102]"
2583,P7,EC17,435,43,373,Language very back strategy.,"[101, 6828, 1304, 1171, 5564, 119, 102]"
2584,P7,EC17,435,43,374,Feel budget tough edge read collection however...,"[101, 14425, 4788, 8035, 2652, 2373, 2436, 164..."
2585,P7,EC17,435,39,310,Car true daughter guy strategy.,"[101, 8185, 2276, 1797, 2564, 5564, 119, 102]"


In [64]:
# size_prefix_ids = len(prefix_ids)

prefix_ids_c = prefix_ids.copy()
full_recfldgrn_name_c = full_recfldgrn_name

In [65]:
df_Rec = dfx[prefix_ids_c + [full_recfldgrn_name_c]]
for i in range(len(prefix_ids_c)):
    
    print(prefix_ids_c, full_recfldgrn_name_c)
    df_Rec, full_recfldgrn_name_c, prefix_ids_c = get_compressed_df(df_Rec, 
                                                                    full_recfldgrn_name_c, 
                                                                    prefix_ids_c)
    print(prefix_ids_c, full_recfldgrn_name_c, '\n')
    
    
    
df_p = df_Rec
df_p

['PID', 'ECID', 'PNID', 'PNSectID'] PNSectSent:Sentence-TkLLMGrn
['PID', 'ECID', 'PNID'] PNSect-PNSectSent:Sentence-TkLLMGrn 

['PID', 'ECID', 'PNID'] PNSect-PNSectSent:Sentence-TkLLMGrn
['PID', 'ECID'] PN-PNSect-PNSectSent:Sentence-TkLLMGrn 

['PID', 'ECID'] PN-PNSect-PNSectSent:Sentence-TkLLMGrn
['PID'] EC-PN-PNSect-PNSectSent:Sentence-TkLLMGrn 

['PID'] EC-PN-PNSect-PNSectSent:Sentence-TkLLMGrn
[] P-EC-PN-PNSect-PNSectSent:Sentence-TkLLMGrn 



,PID,P-EC-PN-PNSect-PNSectSent:Sentence-TkLLMGrn
0,P0,"[[[[[101, 3128, 4504, 1136, 119, 102]], [[101,..."
1,P1,"[[[[[101, 12197, 5363, 170, 1794, 119, 102]], ..."
2,P2,"[[[[[101, 8123, 2134, 1415, 2400, 1696, 1481, ..."
3,P3,"[[[[[101, 3517, 1965, 1770, 1885, 119, 102]], ..."
4,P4,"[[[[[101, 1986, 3083, 1265, 1710, 119, 102]], ..."
5,P5,"[[[[[101, 4659, 1846, 2060, 2862, 5042, 2731, ..."
6,P6,"[[[[[101, 2617, 1988, 3683, 2256, 1946, 1128, ..."
7,P7,"[[[[[101, 8185, 2276, 1797, 2564, 5564, 119, 1..."


In [66]:
import os
import pandas as pd

from recfldgrn.datapoint import convert_PID_to_PIDgroup, RANGE_SIZE, write_df_to_folders, load_df_data_from_folder, get_df_rec


In [67]:
# write_df_to_folders()

In [73]:
df = df_p
tensor_folder = 'data/EHRTensor/'
RecFldGrnName = full_recfldgrn_name_c

fulldata_folder = os.path.join(tensor_folder, RecFldGrnName)
print(df.shape, fulldata_folder)

(8, 3) data/EHRTensor/P-EC-PN-PNSect-PNSectSent:Sentence-TkLLMGrn


In [74]:
write_df_to_folders(RecName, fulldata_folder, df)